# Installing transformers & Importing dependencies

In [1]:
!pip install transformers

     |████████████████████████████████| 2.3MB 27.5MB/s 
     |████████████████████████████████| 901kB 54.2MB/s 
     |████████████████████████████████| 3.3MB 51.1MB/s 


In [2]:
from transformers import pipeline

1. Loading Sumamrization pipeline

In [3]:
summarizer= pipeline ('summarization')

2. Sumamrize text

In [4]:
 response ="""
I had unprotected sex with a man last week and afterwards, he told me that his previous partner has HSV-2. He told me that she hasn’t had an outbreak in years and that he’s never had an outbreak. Should I be concerned about contracting HSV-2? I was going to get tested in a month and see what the results are. Just been feeling pretty depressed about all of this. Any information will help
 """

In [5]:
summarizer(response, max_length=30, min_length= 15, do_sample= False)

[{'summary_text': ' Should I be concerned about contracting HSV-2? I was going to get tested in a month and see what the results are?'}]

In [6]:
response1= """Dear members,

Many of us, including myself, have spent a good bit of time in hospitals, as a patient or a caregiver or both, and have ideas about how hospitals can improve.

Dr. Harlan Krumholz, a cardiologist, healthcare scientist, and health care improvement expert at Yale University, recently asked the following questions on Twitter:

“I am curious to know from patients, especially those recently in the hospital: What do you wish we did better? What were the most obvious ways that quality could be improved throughout hospitalization and in the transition to home? We should all really want to know. What should we improve?”

Harlan’s tweet generated many good replies, and several days ago I asked Harlan what he would think about me posing this question on his behalf here in Inspire. As Harlan is, I’m proud to say, a fan of Inspire, he readily agreed. I invite you to reply to this discussion with answers to the questions he posed, so that we can help to drive positive change in hospitals.

We would like to create a summary from your answers and share them, in an anonymous fashion (no Inspire username), as an essay in a publication, or present the combined insights at a conference. We will let you know of any publication or presentation of this information. Your perspectives matter, and together we can make an impact.

I look forward to reading your thoughts and ideas, and if for any reason you would prefer that we not include your answer in our summary, just write, “Don’t share” in your reply.

Together we're better,
Brian Loew
Founder, Inspire"""


In [7]:
summary_text= summarizer(response1, max_length=240, min_length= 15, do_sample= False)

In [8]:
summary_text[0] #a dictionary

{'summary_text': ' Dr. Harlan Krumholz, a cardiologist, healthcare scientist at Yale University, recently asked the following questions on Twitter: What do you wish we did better? What were the most obvious ways that quality could be improved throughout hospitalization and in the transition to home?'}

In [9]:
summary_text[0]['summary_text']

' Dr. Harlan Krumholz, a cardiologist, healthcare scientist at Yale University, recently asked the following questions on Twitter: What do you wish we did better? What were the most obvious ways that quality could be improved throughout hospitalization and in the transition to home?'

In [10]:
import pandas as pd
dataset_df= pd.read_csv('/content/drive/MyDrive/STD_NLP_Analysis/std_data_finalizing.csv',encoding='latin1')



In [11]:
print(dataset_df.shape)

(574, 4)


In [12]:
print("NUmber of datapoints=  ", dataset_df.shape[0])

NUmber of datapoints=   574


In [13]:
dataset_df.head()

,Name,Profile_url,Description,Time
0,hellomorgan,https://www.inspire.com/m/hellomorgan/,I had unprotected sex with a man last week and...,Yesterday at 4:02 pm
1,sdiv,https://www.inspire.com/m/sdiv/,Im feeling...hopeless. Ive had great strides...,"June 28, 2019 at 3:19 am"
2,Brian,https://www.inspire.com/m/Brian/,"Dear members,\n\nMany of us, including myself,...","March 13, 2019 at 10:24 am"
3,mastergousau2,https://www.inspire.com/m/mastergousau2/,**Attached photo(uncircumcised flaccid penis(f...,"June 19, 2019 at 12:56 pm"
4,khack2471,https://www.inspire.com/m/khack2471/,Since I have had hpv on numerous of occasions ...,"June 19, 2019 at 11:50 pm"


In [14]:
print('Features : ', dataset_df.columns.values)

Features :  ['Name' 'Profile_url' 'Description' 'Time']


In [15]:
dataset_df[dataset_df.isnull().any(axis=1)]

,Name,Profile_url,Description,Time


In [16]:
dataset_df.head()

,Name,Profile_url,Description,Time
0,hellomorgan,https://www.inspire.com/m/hellomorgan/,I had unprotected sex with a man last week and...,Yesterday at 4:02 pm
1,sdiv,https://www.inspire.com/m/sdiv/,Im feeling...hopeless. Ive had great strides...,"June 28, 2019 at 3:19 am"
2,Brian,https://www.inspire.com/m/Brian/,"Dear members,\n\nMany of us, including myself,...","March 13, 2019 at 10:24 am"
3,mastergousau2,https://www.inspire.com/m/mastergousau2/,**Attached photo(uncircumcised flaccid penis(f...,"June 19, 2019 at 12:56 pm"
4,khack2471,https://www.inspire.com/m/khack2471/,Since I have had hpv on numerous of occasions ...,"June 19, 2019 at 11:50 pm"


In [17]:
for index, rows in dataset_df.iterrows():
  if index==0:
    print("index=", index)
    response=(rows['Description'])
    print(len(response))
    print(response)

index= 0
389
I had unprotected sex with a man last week and afterwards, he told me that his previous partner has HSV-2. He told me that she hasnt had an outbreak in years and that hes never had an outbreak. Should I be concerned about contracting HSV-2? I was going to get tested in a month and see what the results are. Just been feeling pretty depressed about all of this. Any information will help


In [18]:
summarizer=pipeline('summarization')

In [19]:


responseSummaryList=[]

for index, rows in dataset_df.iterrows():
  #processing
  patient_response= rows['Description']
  patient_response= patient_response.replace('.','.<eos>')
  patient_response= patient_response.replace('!','!<eos>')
  patient_response= patient_response.replace('?','?<eos>')
  patient_response= patient_response.replace(')',')<eos>')
  
  sentences= patient_response.split('<eos>')
  
  max_chunk= 500
  current_chunk=0
  chunks=[]

  for sentence in sentences:
    if len(chunks)== current_chunk+1:
      if (len(chunks[current_chunk])+ len(sentence.split(' '))) <= max_chunk:
        chunks[current_chunk].extend(sentence.split(' '))

      else:
        current_chunk = current_chunk+1
        chunks.append(sentence.split(' '))

    else:
    #print(current_chunk)
      chunks.append(sentence.split(' '))
  
  #adding the word-components back as a sentence
  for chunk_id in range(len(chunks)):
    chunks[chunk_id]=' '.join(chunks[chunk_id])
  

  #summarizing now
  result= summarizer(chunks, max_length=120, min_length= 30, do_sample= False)

  #concatenate all chunk's summaries
  final_response_summary= ' '.join([elements['summary_text'] for elements in result])

  #appending the final summary into our list
  responseSummaryList.append(final_response_summary)


dataset_df['summary']= responseSummaryList
dataset_df.head()


dataset_df.to_csv(r'/content/drive/MyDrive/STD_NLP_Analysis/summary.csv', index = False)

Your max_length is set to 120, but you input_length is only 98. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 120, but you input_length is only 87. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 120, but you input_length is only 78. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 120, but you input_length is only 52. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 120, but you input_length is only 80. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 120, but you input_length is only 99. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 120, but you input_length is only 32. You might consider

In [20]:
dataset_df.head()

,Name,Profile_url,Description,Time,summary
0,hellomorgan,https://www.inspire.com/m/hellomorgan/,I had unprotected sex with a man last week and...,Yesterday at 4:02 pm,I had unprotected sex with a man last week an...
1,sdiv,https://www.inspire.com/m/sdiv/,Im feeling...hopeless. Ive had great strides...,"June 28, 2019 at 3:19 am",The thought of even meeting a man and having ...
2,Brian,https://www.inspire.com/m/Brian/,"Dear members,\n\nMany of us, including myself,...","March 13, 2019 at 10:24 am","Dr. Harlan Krumholz, a cardiologist, healthca..."
3,mastergousau2,https://www.inspire.com/m/mastergousau2/,**Attached photo(uncircumcised flaccid penis(f...,"June 19, 2019 at 12:56 pm",A 28-year-old male male received his STD tes...
4,khack2471,https://www.inspire.com/m/khack2471/,Since I have had hpv on numerous of occasions ...,"June 19, 2019 at 11:50 pm",Since I have had hpv on numerous of occasions...
